# `QuantumPioneer/databases` Demo
This notebook demonstrates how to interact with the `QuantumPioneer` databases.

See inline comments for more information.

In [1]:
import numpy as np

# both pandas and polars can read parquet files - pick whichever you prefer!
# there are advantages and disadvantages to both
import pandas as pd
import polars as pl

import pyarrow as pa
# this library interacts with the parquet format directly, and both pandas and polars can use it too
import pyarrow.parquet as pq

# schema = layout of the database (what are the datatypes, etc.)
# the schema for the quantumpioneer databases are stored in databases.schema and vary depending on the type of data
from databases.schema import DLPNO_SCHEMA, DFT_SCHEMA

## DLPNO

In [2]:
# set this to match wherever you have the database file located
# DLPNO_DATABASE_FPATH = "QuantumPioneer_v1_DLPNO.parquet"
DLPNO_DATABASE_FPATH = "C:\\VS mk306 TestEnv\\QuantumPioneer\\databases\\dlpno_ts_v5.parquet"

In [3]:
# just open and read the entire dataset (very practical with the DLPNO data, which is small), which will be slow with pandas
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    schema=DLPNO_SCHEMA,  # pandas will try and guess this on its own if you don't provide it - it gets it right, but is slower
)
df.head(4)

,source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au,t1_diagnostic
0,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-552.285416,2401.0,"[[-1.18079, 1.675527, 1.578389], [-0.208623, 2...",1.07331,0.016131
1,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-570.750611,2936.0,"[[2.188448, 0.23618, -1.582947], [1.462597, -0...",1.87994,0.021023
2,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-572.077854,2755.0,"[[1.863694, -1.837741, 0.404006], [1.711148, -...",1.17816,0.021460
3,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-572.156665,2168.0,"[[0.511578, 0.143144, 1.144126], [0.099432, -0...",2.02224,0.015178


In [4]:
# from here you can do all of your usual pandas manipulations
df.iloc[0][["route_section", "input_coordinates"]].to_list()

['uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J cc-pvqz/c cc-pvqz-f12-cabs RIJCOSX VeryTightSCF NormalPNO',
 array([array([-1.18079 ,  1.675527,  1.578389]),
        array([-0.208623,  2.492819,  1.043383]),
        array([-0.779649,  1.343507,  2.394862]),
        array([ 0.39244 ,  2.157912, -2.692596]),
        array([-0.621929,  0.546014, -1.284805]),
        array([-0.971654,  1.756936, -2.170265]),
        array([ 0.817017,  2.041762, -4.123272]),
        array([ 0.68777 ,  3.397201, -3.461238]),
        array([ 0.738187,  1.006404, -0.756896]),
        array([ 1.345064,  1.751076, -1.725896]),
        array([-0.523152, -0.705763, -2.043505]),
        array([-1.337158,  0.408528, -0.460266]),
        array([-1.38121 ,  2.545217, -1.521893]),
        array([-1.689654,  1.529796, -2.967584]),
        array([ 1.825474,  1.662826, -4.301803]),
        array([ 0.068298,  1.737138, -4.858615]),
        array([ 1.610191,  3.922564, -3.20463 ]),
        array([-0.151878,  4.041207, -3.73

In [5]:
# you can reduce the amount of memory consumed by only loading the columns that you care about using columns=...
df = pd.read_parquet(DLPNO_DATABASE_FPATH, columns=["source", "energy"])
df.head(4)

,source,energy
0,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,-552.285416
1,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,-570.750611
2,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,-572.077854
3,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,-572.156665


In [6]:
# and you filter out specific rows _when reading_ the database to further reduce memory consumption (and speed things up)
# these statements can be complex, but the pandas docs explain it well:
# https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    # SKIP rows where...
    filters=[
        [
            (  # multiplicity is equal to 1
                "multiplicity",
                "=",
                1,
            ),  # AND
            (  # energy is less than -500
                "energy",
                "<",
                -500,
            ),
        ],
        [  # OR
            (  # these two specific files
                "source",
                "not in",
                (
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146857.log",
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146989.log",
                ),
            ),
        ],
    ],
)
df.head(4)

,source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au,t1_diagnostic
0,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-552.285416,2401.0,"[[-1.18079, 1.675527, 1.578389], [-0.208623, 2...",1.07331,0.016131
1,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-570.750611,2936.0,"[[2.188448, 0.23618, -1.582947], [1.462597, -0...",1.87994,0.021023
2,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-572.077854,2755.0,"[[1.863694, -1.837741, 0.404006], [1.711148, -...",1.17816,0.021460
3,/data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...,uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...,0,2,-572.156665,2168.0,"[[0.511578, 0.143144, 1.144126], [0.099432, -0...",2.02224,0.015178


In [7]:
# you can also filter out based on one row without actually loading it
df = pd.read_parquet(DLPNO_DATABASE_FPATH, filters=[("run_time", "<", 100)], columns=["source", "charge"])
df.head(4)

,source,charge


In [8]:
# you can run everything from above using polars as well, and in my experience it uses less memory and is faster
df = pl.read_parquet(DLPNO_DATABASE_FPATH)
df.head(4)

source,route_section,charge,multiplicity,energy,run_time,input_coordinates,dipole_au,t1_diagnostic
str,str,u8,u8,f64,u32,list[list[f64]],f32,f32
"""/data1/groups/RMG/Projects/Hao…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-552.285416,2401,"[[-1.18079, 1.675527, 1.578389], [-0.208623, 2.492819, 1.043383], … [-0.436956, -1.690886, -2.642081]]",1.07331,0.016131
"""/data1/groups/RMG/Projects/Hao…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-570.750611,2936,"[[2.188448, 0.23618, -1.582947], [1.462597, -0.505002, -0.684492], … [3.449137, -4.208908, 2.296006]]",1.87994,0.021023
"""/data1/groups/RMG/Projects/Hao…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-572.077854,2755,"[[1.863694, -1.837741, 0.404006], [1.711148, -1.238914, 1.627546], … [4.269185, 1.746003, -0.377551]]",1.17816,0.02146
"""/data1/groups/RMG/Projects/Hao…","""uHF UNO DLPNO-CCSD(T)-F12D cc-…",0,2,-572.156665,2168,"[[0.511578, 0.143144, 1.144126], [0.099432, -0.43431, -0.040568], … [1.178596, -1.353135, 2.218449]]",2.02224,0.015178


In [9]:
# a notable difference is that polars sets `memory_map=True` by default (pandas supports it, but is False and accessible via kwarg only)

In [10]:
# to pass filters to polars, you have to use the `pyarrow_options` argument (polars only supports limiting the number of rows in
# in sequential order via `n_rows`)
df = pl.read_parquet(
    DLPNO_DATABASE_FPATH,
    columns=["source", "charge"],
    pyarrow_options=dict(
        filters=[("run_time", "<", 100)],
        schema=DLPNO_SCHEMA,
    ),
)
df.head(4)

source,charge
str,u8
"""/data1/groups/RMG/Projects/Hao…",0
"""/data1/groups/RMG/Projects/Hao…",0
"""/data1/groups/RMG/Projects/Hao…",0
"""/data1/groups/RMG/Projects/Hao…",0


In [11]:
# ...or just use polars other functions
df = (
    pl.scan_parquet(
        DLPNO_DATABASE_FPATH,
    )  # opens the file, but does not actually read it (LazyFrame)
    .filter(
        pl.col("run_time") < 100,
    )  # sets up our filters, but still does not run the query
    .select(pl.col("source"), pl.col("energy"))
    .collect()  # actually runs the query
)
df.head(4)

source,energy
str,f64


In [12]:
# the final option is to interact with the data via pyarrow directly, which takes all the same arguments as before but in a slightly
# different setup - this is the single fastest way to read the data
table = pq.ParquetDataset(DLPNO_DATABASE_FPATH, schema=DLPNO_SCHEMA, filters=[("run_time", "<", 100)]).read(columns=["source", "energy"])
df = table.to_pandas()
df.head(4)

,source,energy


## DFT
All of the above applies for the DFT data also - this will just demonstrate some of the fields that are specific to the DFT data, as well as limiting the number of rows (since this dataset is much bigger).

This section will just use `pyarrow.parquet` since this dataset is more complicated and this library can do a lot more "nuts and bolts" interactions with the data.
Everything shown could _probably_ be done with pandas or polars, but not as easily.

In [ ]:
# set this to match wherever you have the database file located
DFT_DATABASE_FPATH = "QuantumPioneer_v1_DFT.parquet"
# DLPNO_DATABASE_FPATH = "C:\\VS mk306 TestEnv\\QuantumPioneer\\databases\\dlpno_ts_v5.parquet"

In [ ]:
# look at the information about the database
f = pq.ParquetFile(DFT_DATABASE_FPATH)
f.metadata

In [ ]:
# many rows in this data! just load some of them (see https://stackoverflow.com/a/69888274)
first_ten_rows = next(f.iter_batches(batch_size=64))
df = pa.Table.from_batches([first_ten_rows]).to_pandas()
df.head(4)

In [ ]:
# we can also apply the many filters from above on this data
table = pq.ParquetDataset(
    DFT_DATABASE_FPATH,
    schema=DFT_SCHEMA,
    filters=[("hf", "!=", np.nan)],  # skip rows where hf is missing
    memory_map=True,  # reduce memory usage by delaying reads into memory
).read(columns=["multiplicity", "hf"])

for batch in table.to_batches(512):
    print(batch.to_pandas().head(4))
    break

In [ ]:
# the last thing I will point to is the pyarrow scanner, which makes a lot of these operations easy too
# https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Scanner.html
# This part of the pyarrow API is currently experimental, so it might change, but it is very useful

ds = pa.dataset.dataset(DFT_DATABASE_FPATH, schema=DFT_SCHEMA)
s = pa.dataset.Scanner.from_dataset(
    ds,
    columns=["charge", "multiplicity", "xyz"],
    filter=(pa.compute.field("hf") != pa.compute.scalar(np.nan)),  # must use pyarrow.compute syntax instead of plain strings...
    batch_size=5_096,  # reduce this to fit in your memory limitations
)
for batch in s.to_batches():
    print(batch.to_pandas()['xyz'].head(4))
    break